In [2]:
import pandas as pd
import folium   # (https://pypi.python.org/pypi/folium)
df = pd.read_csv('https://github.com/skickar/CreepDetector/blob/master/session/fuckkk.csv', delimiter = ',', encoding='latin-1', header=0) 
df.sample(10)


ModuleNotFoundError: No module named 'folium'

In [20]:
# MAC , Lon, Lat
cleanedList = []
for i in range (0,(len(df['mac address']))):
    cleanedList.append([[(df['mac address'][i]),(df['lat'][i]),(df['lon'][i])]])
print(cleanedList[1])

[['00:0D:97:08:16:09', 34.197994, -118.34378000000001]]
